# Dow jones
## IBM


In [39]:
import bs4 as bs
import requests
import json
from pymongo import MongoClient
import re

In [ ]:
## Recuperer le ticker 30 valeurs du dow jones 
def dowjones_ticker():
    #scrap yahoo finance for tickers
    tickers_dic={'Nike': 'NKE', 'IBM': 'IBM', "McDonald's Corporation": 'MCD'}
    return tickers_dic


## Recuperation times series sur alphavantage
def get_stock_timeserie_alphavantage(ticker, api_key):
    url = "https://www.alphavantage.co/query?"
    querystring = {"outputsize":"full","datatype":"json","function":"TIME_SERIES_DAILY","symbol":ticker,"apikey":api_key}
    response = requests.request("GET", url, params=querystring)
    stock_json = response.json()
    stock= stock_json['Time Series (Daily)']
    return stock

## Enlever le dot sinon impossible d'inserer dans Mongodb
def sanitize(value, is_value=True):
    if isinstance(value, dict):
        value = {sanitize(k,False):sanitize(v,True) for k, v in value.items()}
    elif isinstance(value, list):
        value = [sanitize(v, True) for v in value]
    elif isinstance(value, str):
        if not is_value:
            value = re.sub(r"[.]", "", value)        
    return value

# rajout de la date dans le dict avec comme key date
def prepare_mongo_documents(stock_json_sanitize):
    stock_json_sanitize_ready= [v for k, v in stock_json_sanitize.items()]
    for i in range(len(st)):   
        stock_json_sanitize_ready[i]['date']= list(stock_json_sanitize)[i]
    return stock_json_sanitize_ready
    
def insert_mongo (name,stock_json_sanitize_ready):
    client = MongoClient('localhost', 27017)
    db = client['DowJonesStocks']
    collection = db[name]
    collection.insert_many(stock_json_sanitize_ready)
    client.close()


def dow_jones_stocks (api_key):
    tickers_dic = dowjones_ticker()
    for k, v in tickers_dic.items():
        stock_json= get_stock_timeserie_alphavantage(v, api_key)
        
        stock_json_sanitize = sanitize(stock_json)
        
        stock_json_sanitize_ready =prepare_mongo_documents(stock_json_sanitize)
        
        insert_mongo(k,stock_json_sanitize_ready)
        
    return None
        
dow_jones_stocks("1OEOQV93R3O4C8D5")              
        


In [5]:
## Recuperer le ticker 30 valeurs du dow jones 

def dowjones_ticker():
    #scrap yahoo finance for tickers
    tickers_dic={'Nike': 'NKE', 'IBM': 'IBM', "McDonald's Corporation": 'MCD'}
    return tickers_dic

dowjones_ticker()

In [208]:
## Recuperation times series sur alphavantage

def get_stock_timeserie_alphavantage(ticker, api_key):
    url = "https://www.alphavantage.co/query?"
    querystring = {"outputsize":"full","datatype":"json","function":"TIME_SERIES_DAILY","symbol":ticker,"apikey":api_key}
    response = requests.request("GET", url, params=querystring)
    stock_json = response.json()
    return stock_json['Time Series (Daily)']


In [209]:

stock_json = get_stock_timeserie_alphavantage("IBM","1OEOQV93R3O4C8D5")
type(stock_json)

dict

In [210]:
stock_json

{'2020-09-08': {'1. open': '122.1600',
  '2. high': '122.8700',
  '3. low': '120.7100',
  '4. close': '121.2100',
  '5. volume': '5210292'},
 '2020-09-04': {'1. open': '124.3500',
  '2. high': '125.3500',
  '3. low': '121.2400',
  '4. close': '122.3000',
  '5. volume': '6018238'},
 '2020-09-03': {'1. open': '128.1900',
  '2. high': '129.9500',
  '3. low': '123.6500',
  '4. close': '124.4500',
  '5. volume': '5716750'},
 '2020-09-02': {'1. open': '123.7200',
  '2. high': '128.7000',
  '3. low': '123.5750',
  '4. close': '128.1800',
  '5. volume': '6592430'},
 '2020-09-01': {'1. open': '122.8500',
  '2. high': '123.9500',
  '3. low': '122.1500',
  '4. close': '123.4000',
  '5. volume': '3155623'},
 '2020-08-31': {'1. open': '125.2500',
  '2. high': '125.2500',
  '3. low': '123.0300',
  '4. close': '123.3100',
  '5. volume': '4827879'},
 '2020-08-28': {'1. open': '124.9600',
  '2. high': '125.3000',
  '3. low': '124.2187',
  '4. close': '125.0700',
  '5. volume': '3099940'},
 '2020-08-27'

In [211]:
#enlever le dot sinon impossible d'inserer dans Mongodb
def sanitize(value, is_value=True):
    if isinstance(value, dict):
        value = {sanitize(k,False):sanitize(v,True) for k, v in value.items()}
    elif isinstance(value, list):
        value = [sanitize(v, True) for v in value]
    elif isinstance(value, str):
        if not is_value:
            value = re.sub(r"[.]", "", value)
            
    return value

stock_json_sanitize = sanitize(stock_json)

stock_json_sanitize

{'2020-09-08': {'1 open': '122.1600',
  '2 high': '122.8700',
  '3 low': '120.7100',
  '4 close': '121.2100',
  '5 volume': '5210292'},
 '2020-09-04': {'1 open': '124.3500',
  '2 high': '125.3500',
  '3 low': '121.2400',
  '4 close': '122.3000',
  '5 volume': '6018238'},
 '2020-09-03': {'1 open': '128.1900',
  '2 high': '129.9500',
  '3 low': '123.6500',
  '4 close': '124.4500',
  '5 volume': '5716750'},
 '2020-09-02': {'1 open': '123.7200',
  '2 high': '128.7000',
  '3 low': '123.5750',
  '4 close': '128.1800',
  '5 volume': '6592430'},
 '2020-09-01': {'1 open': '122.8500',
  '2 high': '123.9500',
  '3 low': '122.1500',
  '4 close': '123.4000',
  '5 volume': '3155623'},
 '2020-08-31': {'1 open': '125.2500',
  '2 high': '125.2500',
  '3 low': '123.0300',
  '4 close': '123.3100',
  '5 volume': '4827879'},
 '2020-08-28': {'1 open': '124.9600',
  '2 high': '125.3000',
  '3 low': '124.2187',
  '4 close': '125.0700',
  '5 volume': '3099940'},
 '2020-08-27': {'1 open': '124.6000',
  '2 high'

In [214]:
# rajout de la date dans le dict avec comme key date
stock_json_sanitize_ready= [v for k, v in stock_json_sanitize.items()]
for i in range(len(st)):   
    stock_json_sanitize_ready[i]['date']= list(stock_json_sanitize)[i]

In [215]:
stock_json_sanitize_ready

[{'1 open': '122.1600',
  '2 high': '122.8700',
  '3 low': '120.7100',
  '4 close': '121.2100',
  '5 volume': '5210292',
  'date': '2020-09-08'},
 {'1 open': '124.3500',
  '2 high': '125.3500',
  '3 low': '121.2400',
  '4 close': '122.3000',
  '5 volume': '6018238',
  'date': '2020-09-04'},
 {'1 open': '128.1900',
  '2 high': '129.9500',
  '3 low': '123.6500',
  '4 close': '124.4500',
  '5 volume': '5716750',
  'date': '2020-09-03'},
 {'1 open': '123.7200',
  '2 high': '128.7000',
  '3 low': '123.5750',
  '4 close': '128.1800',
  '5 volume': '6592430',
  'date': '2020-09-02'},
 {'1 open': '122.8500',
  '2 high': '123.9500',
  '3 low': '122.1500',
  '4 close': '123.4000',
  '5 volume': '3155623',
  'date': '2020-09-01'},
 {'1 open': '125.2500',
  '2 high': '125.2500',
  '3 low': '123.0300',
  '4 close': '123.3100',
  '5 volume': '4827879',
  'date': '2020-08-31'},
 {'1 open': '124.9600',
  '2 high': '125.3000',
  '3 low': '124.2187',
  '4 close': '125.0700',
  '5 volume': '3099940',
  '

In [216]:
client = MongoClient('localhost', 27017)
db = client['DowJonesStocks']
collection_currency = db['IBM']

collection.insert_many(stock_json_sanitize_ready)

client.close()
